In [0]:
# Mask R-CNN code
!git clone https://github.com/matterport/Mask_RCNN

In [0]:
import os

In [0]:
os.chdir('Mask_RCNN')

In [0]:
# Mask R-CNN dependencies
!pip3 install -r requirements.txt
!python setup.py install

# COCO tools
!pip3 install git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI
!pip3 install pycocotools

# Windowing backend
!pip3 install pyqt5

In [0]:
import io
from googleapiclient.http import MediaIoBaseDownload
import shutil

In [0]:
file_id = 'SOMETHING'
fh = io.BytesIO()
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
fh.seek(0)
with open('image3.jpg', 'wb') as f:
    shutil.copyfileobj(fh, f, length=131072)
fh = None
!ls

In [0]:
import skimage.io
import skimage

In [0]:
image = skimage.io.imread('image3.jpg')
skimage.io.imshow(image)

In [0]:
# Root directory of the project
ROOT_DIR = os.path.abspath("./")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library

In [0]:
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize

In [0]:
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

In [0]:
import sys
import random
import math
import numpy as np
import cv2

import xml.etree.ElementTree

https://stackoverflow.com/questions/1912434/how-do-i-parse-xml-in-python

In [0]:
class BeeDataset(utils.Dataset):

    def load_bee(self, dataset_dir, subset):
        """Load a subset of the Balloon dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.add_class("bee", 1, "bee")

        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)
        
        image_dir = os.path.join(dataset_dir, "Images")
        annot_dir = os.path.join(dataset_dir, "Annotations")

        # do something for each xml file in annotation dir
        for file in os.listdir(annot_dir):

          path = os.path.join(root_dir, file)

          if os.path.isfile(path):

            assert file.endswith('.xml')
            
            # parse
            eTree = xml.etree.ElementTree.parse(path)
            root = eTree.getroot()

            # trust the hardcode, hope MIT label me does not change
            filename = root[0].text
            height = int(root[3][0].text)
            width = int(root[3][1].text)
            
            polygons = []

            # trust the hardcode, hope MIT label me does not change
            # the first four are metadata
            for obj in root[4:]:
              shape_attr = {
                  'all_points_x': [],
                  'all_points_y': []}
              # polygon is the last element, first one is metadata
              for point in obj[-1][1:]:
                shape_attr['all_points_x'].append(int(point[0].text))
                shape_attr['all_points_y'].append(int(point[1].text))
              polygons.append(shape_attr)

            self.add_image(
                "bee",
                image_id=filename,  # use file name as a unique image id
                path=os.path.join(image_dir, filename),
                width=width, height=height,
                polygons=polygons)

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a balloon dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "bee":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "bee":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)


In [0]:
%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

# COCO Class names
class_names = ['BG', 'bee']

In [0]:
# Model Configuration Settings
class InferenceConfig(coco.CocoConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

In [0]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

In [0]:
# Which weights to start with?
init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True)

In [0]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=1, 
            layers=r"(mrcnn_class_logits)|(mrcnn_bbox_fc)|(mrcnn_bbox)|(mrcnn_mask)")